In [15]:
version_name =  'sim_or_his_with_concat_moviedb_'
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [16]:
import os

# Get the current working directory
cwd = os.getcwd()

# Navigate two levels up and append the 'data/full_redial/' directory
dir_path = os.path.join(os.path.dirname(os.path.dirname(cwd)), 'data', 'full_redial')
print(dir_path)


/home/pc/Documents/CRS/data/full_redial


In [17]:
import torch 
# import torch
# from torch.utils.data import DataLoader, Dataset

# from transformers import GPT2Config, GPT2Tokenizer, BertModel, BertTokenizer, DistilBertModel, DistilBertTokenizer

# from InductiveAttentionModels import GPT2InductiveAttentionHeadModel
# from SequenceCrossEntropyLoss import SequenceCrossEntropyLoss
import torch
# import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel #, AdamW, get_linear_schedule_with_warmup
# from tqdm import tqdm
import numpy as np


# from nltk.translate import bleu_score
# from nltk.translate.bleu_score import sentence_bleu

from weaviate.classes.query import Filter

train_data = torch.load(dir_path + "/train_data")
test_data = torch.load(dir_path + "/test_data")
movie_db = torch.load(dir_path + "/movie_db_20_50")
# movie_db = torch.load("/home/aac/CRS/data/full_redial/movie_db")

# wcd_url = 'https://nd2trbl2tvk62cnivhtkea.c0.asia-southeast1.gcp.weaviate.cloud'
# wcd_api_key = 'kOvQI8I9Rot8iQAzickNPDjN4fDteeJ7DL8t'

wcd_url = 'https://p3gv4bmxrnkp8smtszii9a.c0.asia-southeast1.gcp.weaviate.cloud'
wcd_api_key = 'Lgk1fpDOvHlCX2yznCcbrfYmEV7Afil2aUlv'

http_host = "nguyenbathiem.tail7bf280.ts.net"
grpc_host = "nguyenbathiem.tail7bf280.ts.net"


In [18]:
def add_like(data):
    new_data = []
    for conv_id, conv in data:
        new_conv = []
        for turn, movie_lst in conv:
            if movie_lst:
                new_conv.append((turn, [(movie_id, 1) for movie_id in movie_lst]))
            else:
                new_conv.append((turn, None))
        new_data.append((conv_id, new_conv))
    return new_data
train_data = add_like(train_data)
test_data = add_like(test_data)
train_data[0]


(0,
 [("B: Hi there, how are you? I'm looking for movie recommendations", None),
  ('A: I am doing okay. What kind of movies do you like?', None),
  ('B: I like animations like [MOVIE_ID] and [MOVIE_ID] I also enjoy [MOVIE_ID] Anything artistic',
   [(84779, 1), (191602, 1), (122159, 1)]),
  ('A: You might like [MOVIE_ID] that was a good movie.', [(165710, 1)]),
  ("B: What's it about?", None),
  ('A: It has Alec Baldwin it is about a baby that works for a company and gets adopted it is very funny',
   None),
  ('B: That seems like a nice comedy Do you have any animated recommendations that are a bit more dramatic? Like [MOVIE_ID] for example I like comedies but I prefer films with a little more depth',
   [(151313, 1)]),
  ('A: That is a tough one but I will remember something [MOVIE_ID] was a good one',
   [(203371, 1)]),
  ("B: Ooh that seems cool! Thanks for the input. I'm ready to submit if you are.",
   None),
  ('A: It is animated, sci fi, and has action Glad I could help', None

In [19]:
def replace_movie_ids(turn_string, movies_lst, movie_db):
    # Split the turn string into parts to isolate [MOVIE_ID] placeholders
    parts = turn_string.split("[MOVIE_ID]")
    new_turn_string = parts[0]

    # Iterate over each part and the corresponding movie ID
    for part, (movie_id, liked) in zip(parts[1:], movies_lst):
        # Get the movie title from movie_db
        movie_title = movie_db[movie_id].split("[SEP]")[0].strip()
        # Append the movie title and the next part of the turn string
        new_turn_string += movie_title + part

    return new_turn_string
    
def add_movie(data, movie_db):
    new_data = []
    for conv in data:
        new_conv = []
        for turn in conv[1]:
            turn_string = turn[0]
            movies_lst = turn[1]
            if movies_lst is not None:
                turn_string = replace_movie_ids(turn_string, movies_lst, movie_db)
            new_conv.append((turn_string, movies_lst))
        new_data.append((conv[0], new_conv))
    return new_data

new_train_data = add_movie(train_data, movie_db)
new_test_data = add_movie(test_data, movie_db)
new_train_data[0]

(0,
 [("B: Hi there, how are you? I'm looking for movie recommendations", None),
  ('A: I am doing okay. What kind of movies do you like?', None),
  ('B: I like animations like the triplets of belleville and waking life I also enjoy mary and max Anything artistic',
   [(84779, 1), (191602, 1), (122159, 1)]),
  ('A: You might like the boss baby that was a good movie.', [(165710, 1)]),
  ("B: What's it about?", None),
  ('A: It has Alec Baldwin it is about a baby that works for a company and gets adopted it is very funny',
   None),
  ('B: That seems like a nice comedy Do you have any animated recommendations that are a bit more dramatic? Like a scanner darkly for example I like comedies but I prefer films with a little more depth',
   [(151313, 1)]),
  ('A: That is a tough one but I will remember something final fantasy: the spirits within was a good one',
   [(203371, 1)]),
  ("B: Ooh that seems cool! Thanks for the input. I'm ready to submit if you are.",
   None),
  ('A: It is animat

In [20]:
def process_movie_db(movie_db):
    new_movie_db = {}
    for k in movie_db.keys():
        movie_db_lst = movie_db[k].split("[SEP]")
        meta_data = "Movie title: " + movie_db_lst[0].strip() + "\n" + "Actors: " + movie_db_lst[1].strip() + "\n" + "Directors: " + movie_db_lst[2].strip() + "\n" + "Genre: " + movie_db_lst[3].strip() + "\n" + "Plot: " + movie_db_lst[4].strip()
        new_movie_db[k] = meta_data
    return new_movie_db
# new_movie_db = process_movie_db(movie_db)
# new_movie_db[34505]

In [21]:
# new_movie_db = torch.load('/home/pc/Documents/CRS/data/full_redial/concat_movie_db')
new_movie_db = process_movie_db(movie_db)
new_movie_db

{75796: "Movie title: headhunter\nActors: Lars Mikkelsen, Charlotte Munck, Burkhard Forstreuter\nDirectors: Rumle Hammerich\nGenre: Thriller\nPlot: Martin Vinge, a 35-year-old former journalist turned headhunter, is hired by N.F. Sieger, 85-year-old CEO of Denmark's largest shipping company and oil empire, to find an alternative heir to the firm, as his son Daniel is no longer the preferred choice. This decision sets off a complex power struggle, entangling Martin in a ruthless game that affects his personal life and relationships.",
 75815: "Movie title: angels in the outfield\nActors: Danny Glover, Brenda Fricker, Tony Danza\nDirectors: William Dear\nGenre: Comedy, Drama, Family\nPlot: Roger, a young boy who has lost his mother, lives apart from his father and finds solace in his friendship with J.P. They share a common interest in the California Angels baseball team, fueling Roger's two primary aspirations: forming a real family and witnessing the Angels win the Pennant.",
 75822: "

In [24]:
import torch
from torch.utils.data import Dataset

class MovieRecDataset(Dataset):
    def __init__(self, data, movie_db):
        self.movie_db = movie_db
        self.samples = []
        
        # Process conversations and extract valid samples
        for conv_id, conversation in enumerate(data):
            # Conversation is a tuple where second element is the list of turns
            user_id = int(conversation[0])
            conv_turns = conversation[1]
            
            # Find all recommender turns with movie recommendations
            for i, (text, movies) in enumerate(conv_turns):
                if movies is not None and text.startswith("A:"):  # This turn has movie mentions
                    # Check if any movie has interest level 1
                    has_recommended_movie = True#= any(interest == 1 for _, interest in movies)
                    
                    if has_recommended_movie:
                        # Get sub-conversation up to this turn
                        sub_conv = conv_turns[:i]
                        
                        # Process the conversation
                        processed_conv = []
                        for turn_text, _ in sub_conv:
                            # Replace speaker tags
                            # turn_text = turn_text.replace("B:", "User:").replace("A:", "Recommender:")
                            processed_conv.append(turn_text)
                        
                        # Get movie recommendations and their interest levels
                        movie_recs = []
                        for movie_id, is_mentioned in movies:
                            # Set interest level
                            if is_mentioned == 1:
                                interest_level = 1  # Liked
                            elif is_mentioned == -1:
                                interest_level = 0  # Disliked
                            else:
                                interest_level = 2  # Neutral/Not specified
                            
                            # Only include if movie exists in movie_db
                            if movie_id in self.movie_db:
                                movie_recs.append((movie_id, interest_level))
                        
                        # Add to samples if there are valid movie recommendations
                        if movie_recs:
                            self.samples.append({
                                'user_id': user_id,
                                'conv_id': conv_id,
                                'conversation': '\n'.join(processed_conv), #+ " " + "[REC]",
                                'recommendations': movie_recs
                            })


        print(f"Created dataset with {len(self.samples)} samples")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        return {
            'user_id': sample['user_id'],
            'conv_id': sample['conv_id'],
            'conversation': sample['conversation'],
            'movie_ids': [rec[0] for rec in sample['recommendations']],
            'liked': [rec[1] for rec in sample['recommendations']]
        }
    
    @staticmethod
    def collate_fn(batch):
        # Sort batch by the length of conversations in descending order
        # batch.sort(key=lambda x: len(x['conversation']), reverse=True)

        user_ids = [sample['user_id'] for sample in batch]

        conv_ids = [sample['conv_id'] for sample in batch]

        # Collect conversations
        conversations = [sample['conversation'] for sample in batch]

        # Collect movie_ids and liked lists
        movie_ids = [sample['movie_ids'] for sample in batch]
        liked = [sample['liked'] for sample in batch]

        return {
            'user_ids': user_ids,
            'conv_ids': conv_ids,
            'conversations': conversations,
            'movie_ids': movie_ids,
            'liked': liked
        }


In [25]:
from torch.utils.data import DataLoader

BATCH_SIZE = 8
# Create dataset
train_dataset = MovieRecDataset(new_train_data, movie_db)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=MovieRecDataset.collate_fn
)
test_dataset = MovieRecDataset(new_test_data, movie_db)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=MovieRecDataset.collate_fn
)
for batch in train_loader:
    print(batch["liked"])
    print(batch["movie_ids"])
    # print(batch["conversations"]) 
    for conv in batch["conversations"]:
        print(conv, "\n")
    break

Created dataset with 29404 samples
Created dataset with 3605 samples
[[1], [1], [1], [1], [1], [1], [1], [1]]
[[165710], [203371], [143189], [143189], [204292], [147598], [184418], [141788]]
B: Hi there, how are you? I'm looking for movie recommendations
A: I am doing okay. What kind of movies do you like?
B: I like animations like the triplets of belleville and waking life I also enjoy mary and max Anything artistic 

B: Hi there, how are you? I'm looking for movie recommendations
A: I am doing okay. What kind of movies do you like?
B: I like animations like the triplets of belleville and waking life I also enjoy mary and max Anything artistic
A: You might like the boss baby that was a good movie.
B: What's it about?
A: It has Alec Baldwin it is about a baby that works for a company and gets adopted it is very funny
B: That seems like a nice comedy Do you have any animated recommendations that are a bit more dramatic? Like a scanner darkly for example I like comedies but I prefer film

In [10]:
def save_model_and_tokenizer(model, tokenizer, save_path):
    """
    Save model and tokenizer with custom tokens
    """
    # Save model
    model.save_pretrained(save_path + "/model")
    
    # Save tokenizer with added tokens
    tokenizer.save_pretrained(save_path + "/tokenizer")
    
    # Save model config
    model.config.save_pretrained(save_path + "/model")

def load_model_and_tokenizer(load_path, device='cuda'):
    """
    Load saved model and tokenizer with custom tokens
    """
    # Load tokenizer with added tokens
    tokenizer = AutoTokenizer.from_pretrained(load_path + "/tokenizer")
    
    # Load model
    model = AutoModel.from_pretrained(load_path + "/model").to(device)
    
    return model, tokenizer
# print()
save_path = os.path.dirname(os.path.dirname(cwd)) + "/src/training/outputs/v1"
# save_model_and_tokenizer(model, tokenizer, save_path)
print(save_path)
if not os.path.exists(save_path):
    # Create the directory
    os.makedirs(save_path)
    print("Directory created:", save_path)
else:
    print("Directory already exists.")

/home/pc/Documents/CRS/src/training/outputs/v1
Directory created: /home/pc/Documents/CRS/src/training/outputs/v1


In [11]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, temperature=0.1, alpha=0.25):
        super().__init__()
        self.temperature = temperature
        self.alpha = alpha
    
    def create_movie_masks_pytorch(self, interest_lists, movie_ids):
        """
        Create mask positives for movies using PyTorch operations.
        
        Args:
            interest_lists: List of lists containing interest indicators
            movie_ids: List of lists containing movie IDs
        
        Returns:
            torch.Tensor: Binary mask tensor of shape (num_conversations, num_unique_movies)
        """
        # Flatten movie_ids and convert to tensor
        movie_ids_flat = torch.tensor([id for sub in movie_ids for id in sub])
        
        # Create masks tensor
        num_conversations = len(interest_lists)
        num_total_movies = len(movie_ids_flat)
        masks = torch.zeros((num_conversations, num_total_movies))
        
        # Create masks for each conversation
        for i in range(num_conversations):
            # Convert current conversation's movie IDs to tensor
            conv_movies = torch.tensor(movie_ids[i])
            
            # Create mask for current conversation using broadcasting
            for movie_id in conv_movies:
                # Find all positions where the movie appears in flattened list
                matches = (movie_ids_flat == movie_id)
                # Set those positions to 1 in the mask
                masks[i][matches] = 1
                
        return masks
    
    def forward(self, conv_hidden, movie_hidden, interest_lists, movie_ids):
        """
        Args:
            conv_hidden: conversation embeddings (B x D)
            movie_hidden: movie embeddings (N x D) where N is total movies across all conversations
            interest_lists: list of lists containing interest values for each conversation
        """
        
        mask_positive = self.create_movie_masks_pytorch(interest_lists, movie_ids).to(conv_hidden.device)
        
        mask_negative = ~mask_positive.bool()
        
        # Compute similarities for all pairs (B x N)
        sim = torch.matmul(conv_hidden, movie_hidden.T) / self.temperature
        # print(sim)
        # First apply numerical stability
        max_sim, _ = torch.max(sim, dim=1, keepdim=True)  # (B x 1)
        sim_stable = sim - max_sim.detach() + 1e-5 # Subtract maximum for numerical stability
        
        # Compute exp similarities with numerical stability
        exp_sim = torch.exp(sim_stable)  # (B x N)
        
        # Compute weights for negative samples (B x N)
        negative_weights = torch.where(
            sim > 0,
            self.alpha * sim + 1,
            torch.ones_like(sim)
        ) * mask_negative#.float()
        
        # Compute weighted positive and negative similarities
        weighted_neg_exp_sim = exp_sim * negative_weights  # (B x N)
        # print("weighted_neg_exp_sim", weighted_neg_exp_sim)
        # Sum negative exponentials for each sample
        sum_neg_exp = torch.sum(weighted_neg_exp_sim, dim=1, keepdim=True)  # (B x 1)
        
        # Compute positive exponentials
        pos_exp_sim = exp_sim * mask_positive.float()  # (B x N)
        # print("pos_exp_sim", pos_exp_sim)
        # Add small epsilon to prevent division by zero
        eps = 1e-8
        
        # Compute log probabilities for each positive pair
        log_probs = torch.where(
            mask_positive.bool(),
            -torch.log((pos_exp_sim + eps) / (pos_exp_sim + sum_neg_exp + eps)),
            torch.zeros_like(sim)
        )
        
        # Count number of positive samples per conversation
        num_positives = torch.sum(mask_positive, dim=1)  # (B)
        
        supervised_contrastive_loss_per_sample = torch.sum(log_probs * mask_positive, dim=1) / num_positives

        supervised_contrastive_loss = torch.mean(supervised_contrastive_loss_per_sample)

        return supervised_contrastive_loss
    
    
def mean_pooling(model_output, attention_mask):
    # return model_output[0][:, 0]
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def rec_token_pooling(model_output, attention_mask, input_ids, rec_token_id):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [12]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
from collections import defaultdict

import os, weaviate
import weaviate
from weaviate.classes.init import Auth
from weaviate.util import generate_uuid5
from weaviate.classes.config import Property, DataType, Tokenization, Configure, VectorDistances
import weaviate.classes as wvc
from weaviate.classes.query import MetadataQuery
class MovieEmbeddingDatabase:
    def __init__(self, device='cuda'):
        self.device = device
        
        # Initialize ChromaDB
        # self.client = weaviate.connect_to_custom(
        #     http_host=http_host,        # Hostname for the HTTP API connection
        #     http_port=443,              # Default is 80, WCD uses 443
        #     http_secure=True,           # Whether to use https (secure) for the HTTP API connection
        #     grpc_host=grpc_host,        # Hostname for the gRPC API connection
        #     grpc_port=10000,              # Default is 50051, WCD uses 443
        #     grpc_secure=True,           # Whether to use a secure channel for the gRPC API connection
        # )
        self.client = weaviate.connect_to_local()
        self.collection_name = version_name + "movie_embeddings"
        self.init_collection(self.collection_name)
        
    def init_collection(self, collection_name):
        print("Deleted collection then creating collection...")
        self.client.connect()
        self.client.collections.delete(collection_name)
        self.client.collections.create(
        collection_name,
        vectorizer_config=Configure.Vectorizer.none(),
        properties=[
            Property(
                name="content",
                data_type=DataType.TEXT,
                tokenization=Tokenization.LOWERCASE,
                index_filterable=True,
                index_searchable=True,
            ),
            Property(
                name="obj_id",
                data_type=DataType.INT,
                # tokenization=Tokenization.LOWERCASE,
                index_filterable=True,
                # index_searchable=True,
            ),
        ],
        inverted_index_config=wvc.config.Configure.inverted_index(
                index_null_state=True,
                index_property_length=True,
                index_timestamps=True,
                stopwords_removals=[],
                bm25_k1=1.2,
                bm25_b=0.75,
            ),
        vector_index_config=Configure.VectorIndex.hnsw(
                distance_metric=VectorDistances.COSINE
            ),
        )
        
    def compute_embedding(self, model, tokenizer, texts):
        encoded = tokenizer(
            texts,
            padding=True,
            truncation=True,
            
            return_tensors='pt',
            max_length=512
        ).to(self.device)
        
        with torch.no_grad():
            output = model(**encoded)
            embeddings = mean_pooling(output, encoded['attention_mask'])
            embeddings = F.normalize(embeddings, p=2, dim=1)
        
        return embeddings.cpu().numpy()
    
    def update_embeddings(self, model, tokenizer, movie_db):
        # Reset collection
        self.init_collection(self.collection_name)
        self.collection = self.client.collections.get(self.collection_name)
        # Process movies in batches
        batch_size = 64
        movie_ids = list(movie_db.keys())
        movie_texts = [movie_db[mid] for mid in movie_ids]
        
        for i in tqdm(range(0, len(movie_ids), batch_size), desc="Updating movie embeddings"):
            batch_ids = movie_ids[i:i + batch_size]
            batch_texts = movie_texts[i:i + batch_size]
            batch_embeddings = self.compute_embedding(model, tokenizer, batch_texts)
            
            data_objects = list()
            for k, content in enumerate(batch_texts):
                # print(type(batch_ids[k]), type(content))
                data_object = wvc.data.DataObject(
                    properties={
                            "content": content,
                            "obj_id": int(batch_ids[k])
                        },
                    uuid=generate_uuid5(content+str(batch_ids[k])),
                    vector=batch_embeddings[k].tolist(),
                )
                data_objects.append(data_object)
            
            self.collection.data.insert_many(data_objects)
            
            # print(type(batch_embeddings[0]))
            # for k, content in enumerate(batch_texts):
            #     data_object = wvc.data.DataObject(
            #         properties={
            #                 "content": content,
            #                 "obj_id": batch_ids[k]
            #             },
            #         uuid=generate_uuid5(content+str(batch_ids[k])),
            #         vector=batch_embeddings[k].tolist(),
            #     )
            #     data_objects.append(data_object)
            
            #     weaviate_batch_size = 32
            #     for _ in range(0, len(batch_texts), weaviate_batch_size):
            #         data_objects = list()
            #         for k, content in enumerate(batch_texts[_:_+weaviate_batch_size]):
            #             data_object = wvc.data.DataObject(
            #                 properties={
            #                         "content": content,
            #                         "obj_id": batch_ids[k]
            #                         # "obj_id": batch_conv_ids[_:_+weaviate_batch_size][k],
            #                         # "user_id": batch_user_ids[_:_+weaviate_batch_size][k],
            #                     },
            #                 # uuid=generate_uuid5(content + str(batch_conv_ids[_:_+weaviate_batch_size][k]) + str(batch_user_ids[_:_+weaviate_batch_size][k])),
            #                 uuid=generate_uuid5(content+str(batch_ids[k])),
            #                 vector=batch_embeddings[k].tolist(),
            #             )
            #             data_objects.append(data_object)
                
            #         self.collection.data.insert_many(data_objects)            
    
    def query_id(self, query_embedding, k=10, search_type = "vector", query = ""):
        collection = self.client.collections.get(self.collection_name)
        result = []
        for i in range(len(query_embedding)):
            if search_type == "vector":
                response = collection.query.near_vector(
                    near_vector=query_embedding[i], # your query vector goes here
                    limit=k,
                    # return_metadata=MetadataQuery(distance=True)
                )
                
            elif search_type == "hybrid":
                response = collection.query.hybrid(
                    query=query,
                    vector=query_embedding[i],
                    alpha=0.5,
                    limit=k,
                )
            result.append([obj.properties["obj_id"] for obj in response.objects])

        return result

In [13]:
class ConvEmbeddingDatabase:
    def __init__(self, device='cuda'):
        self.device = device
        
        # Initialize ChromaDB
        # self.client = weaviate.connect_to_custom(
        #     http_host=http_host,        # Hostname for the HTTP API connection
        #     http_port=443,              # Default is 80, WCD uses 443
        #     http_secure=True,           # Whether to use https (secure) for the HTTP API connection
        #     grpc_host=grpc_host,        # Hostname for the gRPC API connection
        #     grpc_port=10000,            # Default is 50051, WCD uses 443
        #     grpc_secure=True,           # Whether to use a secure channel for the gRPC API connection
        # )
        self.client = weaviate.connect_to_local()

        self.collection_name = version_name + "conv_embeddings"
        self.init_collection(self.collection_name)
        
    def init_collection(self, collection_name):
        print("Deleted collection then creating collection...")
        self.client.connect()
        self.client.collections.delete(collection_name)
        self.client.collections.create(
        collection_name,
        vectorizer_config=Configure.Vectorizer.none(),
        properties=[
            Property(
                name="content",
                data_type=DataType.TEXT,
                tokenization=Tokenization.LOWERCASE,
                index_filterable=True,
                index_searchable=True,
            ),
            Property(
                name="obj_id",
                data_type=DataType.INT,
                # tokenization=Tokenization.LOWERCASE,
                index_filterable=True,
                # index_searchable=True,
            ),
            Property(
                name="user_id",
                data_type=DataType.INT,
                # tokenization=Tokenization.LOWERCASE,
                index_filterable=True,
                # index_searchable=True,
            ),
        ],
        inverted_index_config=wvc.config.Configure.inverted_index(
                index_null_state=True,
                index_property_length=True,
                index_timestamps=True,
                stopwords_removals=[],
                bm25_k1=1.2,
                bm25_b=0.75,
            ),
        vector_index_config=Configure.VectorIndex.hnsw(
                distance_metric=VectorDistances.COSINE
            ),
        )
        
    def compute_embedding(self, model, tokenizer, texts):
        encoded = tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors='pt',
            max_length=512
        ).to(self.device)
        
        with torch.no_grad():
            output = model(**encoded)
            embeddings = mean_pooling(output, encoded['attention_mask'])
            embeddings = F.normalize(embeddings, p=2, dim=1)
        
        return embeddings
    
    def update_embeddings(self, model, tokenizer, conv_db):
        # Reset collection
        self.init_collection(self.collection_name)
        self.collection = self.client.collections.get(self.collection_name)
        # Process movies in batches
        batch_size = 64
        conv_ids = [c['conv_id'] for c in conv_db]
        conv_texts = [c['conversation'] for c in conv_db]
        user_ids = [u['user_id'] for u in conv_db]

        for i in tqdm(range(0, len(conv_ids), batch_size), desc="Updating conversation embeddings"):
            batch_conv_ids = conv_ids[i:i + batch_size]
            batch_user_ids = user_ids[i:i + batch_size]
            batch_texts = conv_texts[i:i + batch_size]
            batch_embeddings = self.compute_embedding(model, tokenizer, batch_texts).cpu().numpy()
            
            weaviate_batch_size = 32
            for _ in range(0, len(batch_texts), weaviate_batch_size):
                data_objects = list()
                for k, content in enumerate(batch_texts[_:_+weaviate_batch_size]):
                    data_object = wvc.data.DataObject(
                        properties={
                                "content": content,
                                "obj_id": int(batch_conv_ids[_:_+weaviate_batch_size][k]),
                                "user_id": int(batch_user_ids[_:_+weaviate_batch_size][k]),
                            },
                        uuid=generate_uuid5(content + str(batch_conv_ids[_:_+weaviate_batch_size][k]) + str(batch_user_ids[_:_+weaviate_batch_size][k])),
                        vector=batch_embeddings[k].tolist(),
                    )
                    data_objects.append(data_object)
            
                self.collection.data.insert_many(data_objects)
            
    
    # def query_id(self, query_embedding, conv_ids, k=10, search_type = "vector", query = ""):
    #     collection = self.client.collections.get(self.collection_name)
    #     result = []
    #     for i in range(len(query_embedding)):
    #         conv_id = conv_ids[i]
    #         filter_conv_id = Filter.by_property("obj_id").equal(conv_id)
    #         if search_type == "vector":
    #             response = collection.query.near_vector(
    #                 near_vector=query_embedding[i], # your query vector goes here
    #                 limit=k,
    #                 filters=filter_conv_id,
    #                 # return_metadata=MetadataQuery(distance=True)
    #             )
                
    #         elif search_type == "hybrid":
    #             response = collection.query.hybrid(
    #                 query=query,
    #                 vector=query_embedding[i],
    #                 alpha=0.5,
    #                 limit=k,
    #                 filters=filter_conv_id,
    #             )
    #         result.append([obj.properties["obj_id"] for obj in response.objects])
    #         # print(result)
    #     return result
    
    def get_sim_conv(self, model, tokenizer, conv_db, conv_ids, query_embeddings, k=2, search_type = "vector", query = ""):
        sim_conv_texts = []
        collection = self.client.collections.get(self.collection_name)
        for conv_id, query_embedding in zip(conv_ids, query_embeddings):
            query_embedding = query_embedding.cpu().detach().numpy()
            filter_conv_id = Filter.by_property("obj_id").not_equal(conv_id)
            response = collection.query.near_vector(
                query_embedding,
                filters=filter_conv_id,
                limit=k
            ).objects
            
            sim_conv_content = [item.properties['content'] for item in response[::-1]]
            sim_conv_texts.append("/n".join(sim_conv_content))

        return sim_conv_texts

    def get_his_conv(self, model, tokenizer, user_ids, conv_ids, query_embeddings, k=2, search_type = "vector", query = ""):
        his_conv_texts = []
        collection = self.client.collections.get(self.collection_name)
        
        for user_id, conv_id, query_embedding in zip(user_ids, conv_ids, query_embeddings):
            query_embedding = query_embedding.cpu().detach().numpy()
            filter_userid = Filter.by_property("user_id").equal(user_id)
            filter_conv_id = Filter.by_property("obj_id").not_equal(conv_id)
            response = collection.query.near_vector(
                query_embedding,
                filters=filter_userid & filter_conv_id,
                limit=k
            ).objects
            
            his_conv_content = [item.properties['content'] for item in response[::-1]]
            his_conv_texts.append("/n".join(his_conv_content))
        return his_conv_texts

    # def get_rand_his_conv(self, model, tokenizer, user_ids, conv_ids, k=2, search_type = "vector", query = ""):
    #     his_conv_texts = []
    #     collection = self.client.collections.get(self.collection_name)
        
    #     for user_id, conv_id in zip(user_ids, conv_ids):
    #         filter_userid = Filter.by_property("user_id").equal(user_id)
    #         filter_conv_id = Filter.by_property("obj_id").not_equal(conv_id)
    #         response = collection.query.fetch_objects(
    #             filters=filter_userid & filter_conv_id,
    #             limit=k
    #         ).objects
    #         if len(response) <=0:
    #             his_conv_texts.append('')
    #         else:
    #             np.random.seed(42)
    #             rand_indices = np.random.randint(len(response), size=k)
    #             his_conv_content = [response[i].properties['content'] for i in rand_indices]
    #             his_conv_texts.append("/n".join(his_conv_content))
    #     return his_conv_texts

In [14]:
def compute_recall(query_results:list, positive_items:list, k_values:list):
    """
    Compute recall@k for multiple k values
    """
    recalls = {k: 0.0 for k in k_values}
    
    total = 0
    for pred_items, pos_items in zip(query_results, positive_items):
        for pos_item in pos_items:
            for k in k_values:
                if pos_item in pred_items[:k]:
                    recalls[k] += 1
            total += 1 
        
    recalls = {k: v / total for k, v in recalls.items()}
    return recalls

def evaluate_model(model, tokenizer, test_loader, movie_embedder, conv_embedder, k_values=[1, 10, 50, 100, 300, 500]):
    model.eval()
    all_positives = []
    all_predictions = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader):
            # Process conversation inputs
            conv_encoded = tokenizer(
                batch["conversations"],
                padding=True,
                truncation=True,
                
                return_tensors='pt'
            ).to(device)
            
            conv_output = model(**conv_encoded)
            conv_hidden = mean_pooling(conv_output, conv_encoded['attention_mask'])
            conv_hidden = F.normalize(conv_hidden, p=2, dim=1)

            topk = 2
            his_conv_texts = conv_embedder.get_his_conv(model, tokenizer, batch['user_ids'], batch['conv_ids'], conv_hidden, k=topk)
            
            if "" in his_conv_texts:
                sim_conv_texts = conv_embedder.get_sim_conv(model, tokenizer, train_dataset, batch['conv_ids'], conv_hidden, k=topk)

            for i in range(len(his_conv_texts)):
                if his_conv_texts[i] == '':
                    his_conv_texts[i] = sim_conv_texts[i]
                    
            encode_texts = [h + '\n' + c for h, c in zip(his_conv_texts, batch["conversations"])]
            conv_encoded = tokenizer(
                encode_texts,
                padding=True,
                truncation=True,
                return_tensors='pt'
            ).to(device)
            
            conv_output = model(**conv_encoded)
            conv_hidden = mean_pooling(conv_output, conv_encoded['attention_mask'])
            conv_hidden = F.normalize(conv_hidden, p=2, dim=1).tolist()
            
            # Get positive movie IDs for each conversation
            positive_movies = []
            for conv_movies, conv_likes in zip(batch["movie_ids"], batch["liked"]):
                # pos_indices = [i for i, like in enumerate(conv_likes) if like == 1]
                pos_indices = [i for i, like in enumerate(conv_likes)]
                pos_movies = [conv_movies[i] for i in pos_indices]
                positive_movies.append(pos_movies)
            
            # Query movie database
            results = movie_embedder.query_id(conv_hidden, k=max(k_values))
            
            predicted_movies = results
            
            all_positives.extend(positive_movies)
            all_predictions.extend(predicted_movies)
    
    # Compute recalls
    recalls = compute_recall(all_predictions, all_positives, k_values)
    return recalls

In [15]:
import torch.nn as nn

num_epochs = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
tokenizer.truncation_side = 'left'

model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2', torch_dtype=torch.float32)
model = model.to(device)

# Initialize movie embedding database
movie_embedder = MovieEmbeddingDatabase(device)
conv_embedder = ConvEmbeddingDatabase(device)

criterion = ContrastiveLoss()
parameters_list = list(model.parameters())
optimizer = AdamW(parameters_list, lr=3e-5, eps=1e-6)

num_training_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=600,
    num_training_steps=num_training_steps
)

best_recall = defaultdict(float)
# Update movie embeddings database
print("Init Updating movie embeddings...")
movie_embedder.update_embeddings(model, tokenizer, movie_db)
conv_embedder.update_embeddings(model, tokenizer, train_dataset)

# Evaluation phase
print("Evaluating model...")
recalls = evaluate_model(model, tokenizer, test_loader, movie_embedder, conv_embedder)
# Print metrics
for k, recall in recalls.items():
    print(f'Recall@{k}: {recall*100:.4f}%')
    if recall > best_recall[k]:
        best_recall[k] = recall

print("\nBest recalls so far:")
for k, recall in best_recall.items():
    print(f'Recall@{k}: {recall*100:.4f}%')

/home/pc/miniforge3/envs/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Deleted collection then creating collection...


/home/pc/miniforge3/envs/myenv/lib/python3.11/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Deleted collection then creating collection...
Init Updating movie embeddings...
Deleted collection then creating collection...


Updating movie embeddings: 100%|██████████| 109/109 [00:56<00:00,  1.94it/s]


Deleted collection then creating collection...


Updating conversation embeddings: 100%|██████████| 460/460 [04:29<00:00,  1.71it/s]


Evaluating model...


100%|██████████| 113/113 [02:51<00:00,  1.52s/it]

Recall@1: 2.2358%
Recall@10: 6.4634%
Recall@50: 13.0894%
Recall@100: 17.4187%
Recall@300: 28.3943%
Recall@500: 35.4472%

Best recalls so far:
Recall@1: 2.2358%
Recall@10: 6.4634%
Recall@50: 13.0894%
Recall@100: 17.4187%
Recall@300: 28.3943%
Recall@500: 35.4472%


In [16]:
def train_model(model, tokenizer, train_loader, test_loader, movie_db, train_db, num_epochs=10, device='cuda'):
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for batch in pbar:
            optimizer.zero_grad()
            # Process conversation inputs
            conv_encoded = tokenizer(
                batch["conversations"],
                padding=True,
                truncation=True,
                
                return_tensors='pt'
            ).to(device)
            
            conv_output = model(**conv_encoded)
            conv_hidden = mean_pooling(conv_output, conv_encoded['attention_mask'])
            conv_hidden = F.normalize(conv_hidden, p=2, dim=1)

            topk = 2
            his_conv_texts = conv_embedder.get_his_conv(model, tokenizer, batch['user_ids'], batch['conv_ids'], conv_hidden, k=topk)
            
            if "" in his_conv_texts:
                sim_conv_texts = conv_embedder.get_sim_conv(model, tokenizer, train_dataset, batch['conv_ids'], conv_hidden, k=topk)

            for i in range(len(his_conv_texts)):
                if his_conv_texts[i] == '':
                    his_conv_texts[i] = sim_conv_texts[i]
            
            encode_texts = [h + '\n' + c for h, c in zip(his_conv_texts, batch["conversations"])]
            
            conv_encoded = tokenizer(
                encode_texts,
                padding=True,
                truncation=True,
                return_tensors='pt'
            ).to(device)
            
            conv_output = model(**conv_encoded)
            conv_hidden = mean_pooling(conv_output, conv_encoded['attention_mask'])
            conv_hidden = F.normalize(conv_hidden, p=2, dim=1)
            
            # Process movie inputs
            movie_ids = [item for sublist in batch["movie_ids"] for item in sublist]
            movie_texts = [movie_db[item] for item in movie_ids]
            
            movie_encoded = tokenizer(
                movie_texts,
                padding=True,
                truncation=True,
                return_tensors='pt'
            ).to(device)
            
            movie_output = model(**movie_encoded)
            movie_hidden = mean_pooling(movie_output, movie_encoded['attention_mask'])
            movie_hidden = F.normalize(movie_hidden, p=2, dim=1)
            
            # Compute loss
            loss = criterion(conv_hidden, movie_hidden, batch["liked"], batch["movie_ids"])
            
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}:')
        print(f'Average loss: {avg_loss:.4f}')
        
        
        # Update movie embeddings database
        print("Updating movie embeddings...")
        movie_embedder.update_embeddings(model, tokenizer, movie_db)
        
        # conv_embedder.update_embeddings(model, tokenizer, train_db)
        
        # Evaluation phase
        print("Evaluating model...")
        recalls = evaluate_model(model, tokenizer, test_loader, movie_embedder, conv_embedder)
        
        # Print metrics
        
        # if all new recall > best recall then save model
        if any(recall > best_recall[k] for k, recall in recalls.items()):
            save_model_and_tokenizer(model, tokenizer, save_path)
            print("Model saved!")
            
        for k, recall in recalls.items():
            print(f'Recall@{k}: {recall*100:.4f}%')
            if recall > best_recall[k]:
                best_recall[k] = recall
        
        print("\nBest recalls so far:")
        for k, recall in best_recall.items():
            print(f'Recall@{k}: {recall*100:.4f}%')
        print()
        
        # write these value into log.log
        with open("log.log", "a") as f:
            f.write(f'Epoch {epoch+1}:\n')
            f.write(f'Average loss: {avg_loss:.4f}\n')
            f.write("Evaluating model...\n")
            for k, recall in recalls.items():
                f.write(f'Recall@{k}: {recall*100:.4f}%\n')
            f.write("\nBest recalls so far:\n")
            for k, recall in best_recall.items():
                f.write(f'Recall@{k}: {recall*100:.4f}%\n')
            f.write("\n")
    

In [17]:
train_model(model, tokenizer, train_loader, test_loader, new_movie_db, train_dataset, 10, device)

Epoch 1/10:   0%|          | 0/919 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 

In [ ]:
model, tokenizer = load_model_and_tokenizer(save_path)
model = model.to(device)
# Initialize movie embedding database
movie_embedder = MovieEmbeddingDatabase(device)
conv_embedder = ConvEmbeddingDatabase(device)

# Update movie embeddings database
print("Init Updating movie embeddings...")
movie_embedder.update_embeddings(model, tokenizer, movie_db)
conv_embedder.update_embeddings(model, tokenizer, train_dataset)

# Evaluation phase
print("Evaluating model...")
recalls = evaluate_model(model, tokenizer, test_loader, movie_embedder, conv_embedder)
# Print metrics
for k, recall in recalls.items():
    print(f'Recall@{k}: {recall*100:.4f}%')
    if recall > best_recall[k]:
        best_recall[k] = recall

print("\nBest recalls so far:")
for k, recall in best_recall.items():
    print(f'Recall@{k}: {recall*100:.4f}%')

In [145]:
movies = {
    75796: "Movie title: headhunter\nActors: Lars Mikkelsen, Charlotte Munck, Burkhard Forstreuter\nDirectors: Rumle Hammerich\nGenre: Thriller\nPlot: Martin Vinge, a 35-year-old former journalist turned headhunter, is hired by N.F. Sieger, 85-year-old CEO of Denmark's largest shipping company and oil empire, to find an alternative heir to the firm, as his son Daniel is no longer the preferred choice. This decision sets off a complex power struggle, entangling Martin in a ruthless game that affects his personal life and relationships.",
 75815: "Movie title: angels in the outfield\nActors: Danny Glover, Brenda Fricker, Tony Danza\nDirectors: William Dear\nGenre: Comedy, Drama, Family\nPlot: Roger, a young boy who has lost his mother, lives apart from his father and finds solace in his friendship with J.P. They share a common interest in the California Angels baseball team, fueling Roger's two primary aspirations: forming a real family and witnessing the Angels win the Pennant.",
 75822: "Movie title: eddie and the cruisers\nActors: Tom Berenger, Michael Pare, Joe Pantoliano\nDirectors: Martin Davidson\nGenre: Drama, Music, Mystery\nPlot: In 1964, Eddie Wilson, lead singer of Eddie and the Cruisers, mysteriously disappeared after his car went off a Jersey bridge. Twenty years later, renewed interest in the band sparks a mystery: what if Eddie is still alive? A series of events, including ransacked homes and a never-released album, fuels journalist Maggie Foley's investigation into the band's troubled past.",
 75828: 'Movie title: ninja assassin\nActors: Rain, Rick Yune, Naomie Harris\nDirectors: James McTeigue\nGenre: Action, Thriller\nPlot: Raizo, a skilled assassin trained by the Ozunu clan since childhood, has turned against his former mentors, seeking revenge for their brutal murders. He teams up with Europol investigator Mika, methodically eliminating enemies as he closes in on a long-awaited confrontation with his former master.',
 75867: "Movie title: orgazmo\nActors: Trey Parker, Dian Bachar, Robyn Lynne Raab\nDirectors: Trey Parker\nGenre: Comedy\nPlot: Joe Young, a young Mormon, inadvertently lands a lead role in the film Orgazmo, a sex superhero movie, after impressing the director, Maxxx Orbison, with his fighting skills. Joe takes the job to fund his wedding, hiding his new role from his fiancée. The film's success and a real-life Orgazmorator created by his co-star Ben put Joe in a difficult situation.",
 75881: 'Movie title: toy story 2\nActors: Tom Hanks, Tim Allen, Joan Cusack\nDirectors: John Lasseter, Ash Brannon, Lee Unkrich\nGenre: Animation, Adventure, Comedy\nPlot: While Andy is away at summer camp, Woody is kidnapped by Al McWiggin, a greedy collector who owns "Al\'s Toy Barn." Buzz and his friends, including Mr. Potato Head, Slinky Dog, and Rex, embark on a rescue mission to save Woody from being sold to a museum in Japan, where he may be lost forever.',
 75903: "Movie title: a star is born\nActors: Judy Garland, James Mason, Jack Carson\nDirectors: George Cukor\nGenre: Drama, Musical, Romance\nPlot: Norman Maine, a struggling movie star, meets showgirl Esther Blodgett after stumbling into her act while intoxicated. A friendship develops into romance, but their relationship is strained as Esther's career flourishes while Norman's continues to decline, leading to increasing tensions between the couple.",
 75918: 'Movie title: pretty in pink\nActors: Molly Ringwald, Jon Cryer, Harry Dean Stanton\nDirectors: Howard Deutch\nGenre: Comedy, Drama, Romance\nPlot: Andie, a relatively unpopular high school student, spends time with her friends Iona and Duckie. Duckie has long harbored feelings for Andie, but she has recently caught the attention of Blane, a popular and wealthy student, raising the question of whether their different social circles can intersect.',
 75962: "Movie title: back to hannibal: the return of tom sawyer and huckleberry finn\nActors: Raphael Sbarge, Mitchell Anderson, Megan Follows\nDirectors: Paul Krasny\nGenre: Adventure, Family\nPlot: In the continuing story of Tom Sawyer and Huckleberry Finn, Jim Watson, a former slave, discovers the lifeless body of Lyle Newman, leading townsfolk to accuse him of murder. Jim seeks help from Huckleberry Finn, now a reporter, and Tom Sawyer, now a lawyer, to defend him. They return to Hannibal for Jim's trial, where they face a chilly reception due to Lyle's respected status and connection to Becky Thatcher's husband.",
 75968: 'Movie title: an american werewolf in paris\nActors: Tom Everett Scott, Julie Delpy, Vince Vieluf\nDirectors: Anthony Waller\nGenre: Comedy, Fantasy, Horror\nPlot: A young woman, the daughter of a werewolf from the first film, lives in Paris with her mother and stepfather, who are trying to manage her lycanthropic condition. A trio of American tourists intervene when she attempts to jump from the Eiffel Tower, drawing them into a plot involving a secret werewolf society and a drug that enables werewolves to transform at will, independent of the full moon.',
 75981: "Movie title: the golden temple\nActors: Mike Wells, Apostle Ben, Sue Jackson\nDirectors: Enrico Masi, Stefano Croci\nGenre: Documentary\nPlot: The Olympic regeneration in East London is a metaphor for the contemporary age, likened to filling the sea with sand. The area is a complex, multifaceted space with a stadium, Europe's largest mall, and a small, unconventional church in an old industrial unit. It's a place where people live, work, and interact, reflecting diverse perspectives on austerity, luxury, and community.",
 75984: "Movie title: fool's gold\nActors: Matthew McConaughey, Kate Hudson, Donald Sutherland\nDirectors: Andy Tennant\nGenre: Action, Adventure, Comedy\nPlot: Benjamin Finnegan, a deep-sea treasure hunter, is on the cusp of a century-defining discovery near an island owned by a ruthless rapper, to whom he owes a debt. Broke and recently divorced from his research partner Tess, Ben must convince Nigel Honeycutt to fund his search, win Tess's cooperation, evade the rival treasure hunter, and uncover the centuries-old Spanish treasure.",

}
conv = """B: Hi there, how are you? I'm looking for movie recommendations
A: I am doing okay. What kind of movies do you like?
B: I like animations like the triplets of belleville and waking life I also enjoy mary and max Anything artistic"""
# conv = """B: Hi there, how are you? I'm looking for movie recommendations"""
conv = """B: Hi there, how are you? I'm looking for movie recommendations
A: I am doing okay. What kind of movies do you like?
B: I like animations like the triplets of belleville and waking life I also enjoy mary and max Anything artistic
A: You might like the boss baby that was a good movie.
B: What's it about?
A: It has Alec Baldwin it is about a baby that works for a company and gets adopted it is very funny
B: That seems like a nice comedy Do you have any animated recommendations that are a bit more dramatic? Like a scanner darkly for example I like comedies but I prefer films with a little more depth 
"""

In [146]:
prompt = """You are a movie recommendation system. Based on the user’s conversation and available movie data, generate a response and reorder all movie IDs in movies_id_list to present the most suitable movies.
If the user’s intent is unrelated to movies or there’s insufficient information:
•Respond naturally by answering or asking for more details.
•Set movies_id_list to null.
Movies_id_list:
•Ensure movies_id_list contains all movie IDs reordered from highest suitable to lowest, not just related ones.
•Number of movies in movies_id_list should be equal to the number of movies in the provided list.
Response:
•Make it concise, engaging, and satisfying for the user.
•Only include movies from the provided list; do not create new entries and repeat the movie mentioned in conversation.
Output Format:
{
    "response": "<your response here>",
    "movies_id_list": ["movie_id1", "movie_id2", "..."] or null
}""" + f"""Here is movies id list (id:information):
{json.dumps(movies)}"""
print(prompt)

You are a movie recommendation system. Based on the user’s conversation and available movie data, generate a response and reorder all movie IDs in movies_id_list to present the most suitable movies.
If the user’s intent is unrelated to movies or there’s insufficient information:
•Respond naturally by answering or asking for more details.
•Set movies_id_list to null.
Movies_id_list:
•Ensure movies_id_list contains all movie IDs reordered from highest suitable to lowest, not just related ones.
•Number of movies in movies_id_list should be equal to the number of movies in the provided list.
Response:
•Make it concise, engaging, and satisfying for the user.
•Only include movies from the provided list; do not create new entries and repeat the movie mentioned in conversation.
Output Format:
{
    "response": "<your response here>",
    "movies_id_list": ["movie_id1", "movie_id2", "..."] or null
}Here is movies id list (id:information):
{"75796": "Movie title: headhunter\nActors: Lars Mikkels

In [153]:
from openai import OpenAI
import json


TOGETHER_API_KEY = ""
OPENAI_API_KEY = ""

client = OpenAI(
    api_key=TOGETHER_API_KEY,
    base_url="https://api.together.xyz/v1",
)
openai_client = OpenAI(api_key = OPENAI_API_KEY)
default_model = "meta-llama/Llama-3.2-3B-Instruct-Turbo"
default_model = "gpt-4o-mini"
def infer_chat(chat, model=default_model, max_tokens=1024):
    for i in range(5):
        try:
            chat_response = openai_client.chat.completions.create(
                model=model,
                messages=chat,
                # max_tokens=max_tokens,
                stream=False,
                temperature=0,
            )

            return chat_response.choices[0].message.content
        except Exception as e:
            print(e)
            continue
test_chat = [
        {
            "role": "user",
            "content": prompt,
        },
        {"role": "user", "content": conv},
]  
print(infer_chat(test_chat))

{
    "response": "If you're looking for animated films with more depth, I recommend checking out 'Toy Story 2.' It beautifully explores themes of friendship and loyalty while delivering both humor and emotional moments. Another option is 'A Star is Born,' which, while not animated, offers a deep and dramatic narrative that you might appreciate. Let me know if you'd like more suggestions!",
    "movies_id_list": ["75881", "75903", "75815", "75867", "75918", "75796", "75828", "75968", "75962", "75984"]
}
